#  Model 1

In [1]:
import numpy as np
from keras.layers import Embedding, Reshape, Merge
from keras.models import Sequential

class CFModel(Sequential):

    # The constructor for the class
    def __init__(self, n_users, m_items, k_factors, **kwargs):
        # P is the embedding layer that creates an User by latent factors matrix.
        # If the intput is a user_id, P returns the latent factor vector for that user.
        P = Sequential()
        P.add(Embedding(n_users, k_factors, input_length=1))
        P.add(Reshape((k_factors,)))

        # Q is the embedding layer that creates a Movie by latent factors matrix.
        # If the input is a movie_id, Q returns the latent factor vector for that movie.
        Q = Sequential()
        Q.add(Embedding(m_items, k_factors, input_length=1))
        Q.add(Reshape((k_factors,)))

        super(CFModel, self).__init__(**kwargs)
        
        # The Merge layer takes the dot product of user and movie latent factor vectors to return the corresponding rating.
        self.add(Merge([P, Q], mode='dot', dot_axes=1))

    # The rate function to predict user's rating of unrated items
    def rate(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]


Using TensorFlow backend.


ImportError: cannot import name 'Merge'

## Loading Datasets
Loaded the 3 datasets into 3 dataframes: *ratings*, *users*, and *movies*. 
Set *max_userid* as the max value of user_id in the ratings and *max_movieid* as the max value of movie_id in the ratings.

In [ ]:
# Import libraries
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras

# Reading ratings file
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['user_id'].drop_duplicates().max()
max_movieid = ratings['movie_id'].drop_duplicates().max()

# Reading ratings file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading ratings file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

'''
# Randomly sample 1% of the ratings dataset
small_ratings = ratings.sample(frac=0.02)
small_users = max_userid.sample(frac=0.02)
small_movies = max_movieid.sample(frac=0.02)

# Randomly sample 20% of the ratings dataset
small_ratings = ratings.sample(frac=0.20)
small_users = max_userid.sample(frac=0.20)
small_movies = max_movieid.sample(frac=0.20)

# Randomly sample 60% of the ratings dataset
small_ratings = ratings.sample(frac=0.60)
small_users = max_userid.sample(frac=0.60)
small_movies = max_movieid.sample(frac=0.60)
'''

# Create training set
shuffled_ratings = ratings.sample(frac=1.)

# Shuffling users
Users = shuffled_ratings['user_emb_id'].values
print ('Users:', Users, ', shape =', Users.shape)

# Shuffling movies
Movies = shuffled_ratings['movie_emb_id'].values
print ('Movies:', Movies, ', shape =', Movies.shape)

# Shuffling ratings
Ratings = shuffled_ratings['rating'].values
print ('Ratings:', Ratings, ', shape =', Ratings.shape)


In [ ]:
# Import Keras libraries
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
import numpy as np
import keras
from keras.layers import Embedding, Reshape, Merge
from keras.models import Sequential
from keras.layers import Merge

# Define constants
K_FACTORS = 100 # The number of dimensional embeddings for movies and users
TEST_USER = 2000 # A random test user (user_id = 2000)

# Define model
model = CFModel(max_userid, max_movieid, K_FACTORS)

# Compile the model using MSE as the loss function and the AdaMax learning algorithm
model.compile(loss='mse', optimizer='adamax')

# Callbacks monitor the validation loss
# Save the model weights each time the validation loss has improved
callbacks = [EarlyStopping('val_loss', patience=2), ModelCheckpoint('weights.h5', save_best_only=True)]

# Use 30 epochs, 90% training data, 10% validation data 
history = model.fit([Users, Movies], Ratings, nb_epoch=30, validation_split=.1, verbose=2, callbacks=callbacks)

# Use 100 epochs, 90% training data, 10% validation data 
#history = model.fit([Users, Movies], Ratings, nb_epoch=100, validation_split=.1, verbose=2, callbacks=callbacks)

# Show the best validation RMSE
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print 'Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss))

# Use the pre-trained model
trained_model = CFModel(max_userid, max_movieid, K_FACTORS)

# Load weights
trained_model.load_weights('weights.h5')

# Model 2

In [ ]:
import numpy as np
import time

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D
from keras import optimizers


from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from keras.utils import np_utils

class Collaborative_Filtering_Neural_Net(object):

	def __init__(self, train_data, val_data, mask, num_layers=3, learn_rate=.2):

		self.train_data = train_data
		self.val_data   = val_data
		self.mask       = mask
		self.num_layers = num_layers

		self.m          = self.train_data.shape[0]
		self.n 			= self.train_data.shape[1]
		
		self.learn_rate = learn_rate

		self.construct_input()


	def construct_input(self):
		'''
		Construct training input/output from the training data matrix
		and 
		Construct validation input/output from the training/validation 
		'''
		def change_to_one_hot(value, value_range):
			one_hot_vec = np.zeros(len(value_range))
			one_hot_vec[int(value/.5)] = 1
			return one_hot_vec


		m = self.m
		n = self.n

		user_indices, movie_indices = (np.where(self.train_data > 0))
		scores = self.train_data[self.mask]

		num_train_samples = user_indices.shape[0]

		self.train_x = np.zeros((num_train_samples, m+n))
		self.train_y = np.zeros((num_train_samples, 11))

		start = time.time()

		#construct training input and output X, y
		for i in range(num_train_samples):
			u_ind = user_indices[i]
			m_ind = movie_indices[i]

			self.train_x[i, u_ind]   = 1
			self.train_x[i, m+m_ind] = 1

			score 			= self.train_data[u_ind, m_ind]
			self.train_y[i] = change_to_one_hot(score, np.arange(0,5.5,.5))



		#construct test inputs for where we need to predict values
		user_indices, movie_indices = np.where(self.mask)
		num_test_samples = user_indices.shape[0]
		self.test_x = np.zeros((num_test_samples, m+n))
		self.test_y = np.zeros((num_test_samples, 11))

		for i in range(num_test_samples):
			u_ind = user_indices[i]
			m_ind = movie_indices[i]

			self.test_x[i, u_ind]   = 1
			self.test_x[i, m+m_ind] = 1

			score 		   = self.val_data[u_ind, m_ind]
			self.test_y[i] = change_to_one_hot(score, np.arange(0,5.5,.5))

		print(time.time() - start)


	def construct_model(self, hidden_layer_pattern = 'exponential'):
		'''
		Constructs a Neural network with a given pattern.
		The pattern indicates how many neurons should exist at every layer.
		Param:
			hidden_layer_pattern - The input layer and output layer are fixed, but the rate at which the layer sizes
			decreases depends on the parameter, hidden_layer_pattern
		'''
		model = Sequential()
		input_size = self.m + self.n
		
		# add the first layer
		model.add(Dense(input_size, activation='relu', input_shape=(input_size,)))

		#one of the two model architectures tested
		if (hidden_layer_pattern == 'linear'):
			linear_decrease = int(input_size/self.num_layers)
			for i in range(self.num_layers):
				input_size = input_size - linear_decrease
				model.add(Dense(input_size, activation='relu') )

		if (hidden_layer_pattern == 'exponential'):
			exponential_decrease = int((np.exp(np.log(input_size)/(self.num_layers+2))))
			print(exponential_decrease)
			for i in range(self.num_layers):
				input_size = int(input_size/exponential_decrease);
				model.add(Dense(input_size, activation='relu') )

		print (model.output_shape)
		#one hot encoded output
		model.add(Dense(11, activation='relu'))


		# model says they optimized the log loss error

		adam = optimizers.Adam(lr=self.learn_rate, decay=.001)
		model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

		self.model = model

	def train_model(self, model_number = 0):
		'''
		Trains the model. Saves checkpoints of the model at every epoch.
		I personally just stop training when I find that the loss function has barely changed. Since it takes
		so long to perform each epoch on my computer, I just keep running a 20 epoch train, stop it when I
		have to, then train again later.
		Param:
			model_number - Just changes the filename that the model is saved to. 
						   Don't want to overwrite good save files during training, do you?

		Note: these checkpoints are 1GB each.
		'''
		# lets make checkpoints
		filepath = "nn_model_{}_lr_{}".format(model_number,self.learn_rate)
		filepath+= "_{epoch:02d}.hdf5"

		print('learn_rate = {}'.format(self.learn_rate))
		checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
		callbacks_list = [checkpoint]

		self.model.fit(self.train_x, self.train_y, batch_size=128, epochs=20, callbacks=callbacks_list, verbose=1)

	def load_model(self, filename):
		'''
		Loads the weights of an identically architectured neural net at the given filepath
		'''
		self.model.load_weights(filename)
		adam = optimizers.Adam(lr=self.learn_rate, decay=.001)
		self.model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])



	def predict_values(self, test_type='validation'):
		'''
		Predicts values based on training or validation data
		Return:
			scores
			predicted values
		'''
		# print(self.model.get_weights())
		if (test_type == 'validation'):
			scores = self.model.predict(self.test_x, verbose=True)
			return scores, self.test_y
		elif (test_type == 'training'):
			scores = self.model.predict(self.train_x, verbose=True)
			return scores, self.train_y


